In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# 🔹 1. CARGA DE DATOS
df = pd.read_excel('datos_modelo_GRD.xlsx')  # reemplaza con el nombre real de tu CSV

# 🔹 2. IMPUTACIÓN DE VALORES NULOS
df.fillna(df.mean(numeric_only=True), inplace=True)
df.fillna('Desconocido', inplace=True)  # para variables categóricas

# 🔹 3. FILTRO: variables disponibles antes del alta médica
vars_usables = [
    'Aseguradora -Código', 'Edad', 'Grupo Edad', 'Sexo', 'Codigo de ciudad',
    'Fecha de ingreso', 'Tipo de ingreso', 'Dx de ingreso',
    'Infecciones', 'Infección Quirúrgica'
]
X = df[vars_usables]

# 🔹 4. VARIABLE OBJETIVO
y = df['Tipo GRD']  # puedes cambiarlo a 'GRD -Código' si lo prefieres

# 🔹 5. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
X = pd.get_dummies(X)  # one-hot encoding general
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# 🔹 6. DIVISIÓN Y ESCALADO
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, stratify=y_categorical, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 🔹 7. DEFINICIÓN Y ENTRENAMIENTO DEL MEJOR MODELO DE RED NEURONAL
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=60, batch_size=32, validation_split=0.2, verbose=1)

# 🔹 8. EVALUACIÓN
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nReporte de clasificación:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

# 🔹 9. MATRIZ DE CONFUSIÓN
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicción")
plt.ylabel("Etiqueta real")
plt.title("Matriz de Confusión - Red Neuronal")
plt.show()

# 🔹 10. CURVAS ROC
y_test_bin = label_binarize(y_true, classes=range(len(le.classes_)))
plt.figure(figsize=(8,6))
for i in range(len(le.classes_)):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{le.classes_[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.title('Curvas ROC - Red Neuronal')
plt.xlabel('Falsos positivos')
plt.ylabel('Verdaderos positivos')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'tensorflow'